In [66]:
import pandas as pd

kp_indices = pd.read_csv('kp_indices.csv')
fc1 = pd.read_csv('fc1.csv')

kp_values = []

# Iterate through each row in fc1
for index, row in fc1.iterrows():
    timestamp = pd.Timestamp(row['time'])
    date = timestamp.date()
    hour = timestamp.hour

    # Find the corresponding date row in kp_indices
    kp_row = kp_indices[kp_indices['Date'] == str(date)]

    # Check if kp_row is empty (no matching date)
    if not kp_row.empty:
        # Determine the KP value based on the hour range
        if 0 <= hour < 3:
            kp_value = kp_row['KP-0'].values[0]
        elif 3 <= hour < 6:
            kp_value = kp_row['KP-3'].values[0]
        elif 6 <= hour < 9:
            kp_value = kp_row['KP-6'].values[0]
        elif 9 <= hour < 12:
            kp_value = kp_row['KP-9'].values[0]
        elif 12 <= hour < 15:
            kp_value = kp_row['KP-12'].values[0]
        elif 15 <= hour < 18:
            kp_value = kp_row['KP-15'].values[0]
        elif 18 <= hour < 21:
            kp_value = kp_row['KP-18'].values[0]
        else:
            kp_value = kp_row['KP-21'].values[0]
    else:
        kp_value = 0  # Use 0 as a default value in case of missing data.

    # Append the calculated KP value to the list
    kp_values.append(kp_value)

# Add the 'KP' column to fc1 with the calculated KP values
fc1['KP'] = kp_values


In [67]:
fc1.sample(n=10).head(10)

,time,proton_speed,proton_density,proton_temperature,KP
24622,2023-10-06 21:21:06.396000000,382.4,14.40,21999.0,3.33
20379,2023-10-06 14:18:51.140999936,402.6,7.91,56464.0,1.33
21495,2023-10-06 16:08:43.392000000,414.3,5.20,50496.0,1.67
23578,2023-10-06 19:31:16.888999936,402.9,3.80,147423.0,1.00
9064,2023-10-05 17:00:46.619000064,463.6,5.22,185343.0,1.00
8841,2023-10-05 16:37:09.648999936,452.0,5.17,204390.0,1.00
17186,2023-10-06 08:35:31.793999872,429.4,16.06,52793.0,2.33
9021,2023-10-05 16:56:14.608999936,462.1,5.07,203181.0,1.00
35397,2023-10-07 15:07:43.019000064,477.4,3.34,194034.0,0.33
15316,2023-10-06 04:55:30.091000064,454.9,11.08,117715.0,2.67


In [68]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error

X = fc1[['proton_speed', 'proton_density', 'proton_temperature']]
y = fc1['KP']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', random_state=42, max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f'Test Mean Absolute Error: {mae:.2f}')

fc1_features = scaler.transform(X)  # Standardize the entire fc1 dataset
predicted_kp = model.predict(fc1_features)

fc1['Predicted_KP'] = predicted_kp

Test Mean Absolute Error: 0.38


In [69]:
tolerance=1
accurate_predictions = np.abs(y_test - y_pred) <= tolerance
percentage_accuracy = (np.sum(accurate_predictions) / len(y_test)) * 100.0

print(f'Percentage Accuracy within {tolerance} units: {percentage_accuracy:.2f}%')

Percentage Accuracy within 1 units: 92.10%
